In [3]:
!pip install paddle2onnx onnxruntime onnxslim onnxruntime-tools onnx pickleshare -i https://pypi.tuna.tsinghua.edu.cn/simple

In [15]:
!paddle2onnx --model_dir ../rapid_table_det_paddle/models --model_filename obj_det_paddle.pdmodel --params_filename obj_det_paddle.pdiparams --save_file ../rapid_table_det/models/obj_det.onnx --opset_version 16 --enable_onnx_checker True
!paddle2onnx --model_dir ../rapid_table_det_paddle/models --model_filename edge_det_paddle.pdmodel --params_filename model.pdiparams --save_file ../rapid_table_det/models/edge_det.onnx --opset_version 16 --enable_onnx_checker True
!paddle2onnx --model_dir ../rapid_table_det_paddle/models --model_filename cls_det_paddle.pdmodel --params_filename cls_det_paddle.pdiparams --save_file ../rapid_table_det/models/cls_det.onnx --opset_version 16 --enable_onnx_checker True

!onnxslim ../rapid_table_det/models/obj_det.onnx ../rapid_table_det/models/obj_det.onnx
!onnxslim ../rapid_table_det/models/edge_det.onnx ../rapid_table_det/models/edge_det.onnx
!onnxslim ../rapid_table_det/models/cls_det.onnx ../rapid_table_det/models/cls_det.onnx

In [6]:
from pathlib import Path
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType, quant_pre_process
def quantize_model(root_dir_str, model_dir, pre_fix):

    original_model_path = f"{pre_fix}.onnx"
    quantized_model_path = f"{pre_fix}_quantized.onnx"
    # quantized_model_path = original_model_path
    original_model_path = f"{root_dir_str}/{model_dir}/{original_model_path}"
    quantized_model_path = f"{root_dir_str}/{model_dir}/{quantized_model_path}"
    quant_pre_process(original_model_path, quantized_model_path, auto_merge=True)
    # 进行动态量化
    quantize_dynamic(
        model_input=quantized_model_path,
        model_output=quantized_model_path,
        weight_type=QuantType.QUInt8
    )

    # 验证量化后的模型
    quantized_model = onnx.load(quantized_model_path)
    onnx.checker.check_model(quantized_model)
    print("Quantized model is valid.")

In [7]:
root_dir_str = ".."
model_dir = f"rapid_table_det/models"
quantize_model(root_dir_str, model_dir,  "obj_det")
quantize_model(root_dir_str, model_dir,  "edge_det")

In [16]:
!python fix_onnx2.py